In [131]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import scipy.stats as stats

In [108]:
df = pd.read_csv('dane.csv', sep=';', index_col=0)

In [109]:
df.head(10)

,Cel,Aktywa,Płeć,Stan cywilny,Mieszkanie,Praca,Okres,Wiek,Ocena
1,Drobne AGD,Bardzo niskie,M,Samotna(y),Własne,Fizyczny,13,23,Dobra
2,Meble,Niskie,M,Rozwiedziona(y),Własne,Umysłowy,25,32,Zła
3,Nowy samochód,Bardzo niskie,M,Samotna(y),Własne,Umysłowy,19,38,Zła
4,Meble,Bardzo niskie,M,Samotna(y),Własne,Fizyczny,13,36,Zła
5,Edukacja,Niskie,M,Samotna(y),Wynajem,Umysłowy,40,31,Dobra
6,Meble,Brak,M,Stały związek,Własne,Umysłowy,11,25,Dobra
7,Nowy samochód,Bardzo niskie,M,Stały związek,Własne,Fizyczny,13,26,Dobra
8,Business,Bardzo niskie,M,Samotna(y),Własne,Fizyczny,14,27,Dobra
9,Drobne AGD,Bardzo niskie,M,Samotna(y),Własne,Umysłowy,37,25,Zła
10,Drobne AGD,Brak,K,Rozwiedziona(y),Własne,Umysłowy,25,43,Zła


In [110]:
# Kodowanie zero jedynkowo
df['Płeć'] = df['Płeć'].apply(lambda x: 1 if x == 'M' else 0)

df['Stan cywilny'] = df['Stan cywilny'].apply(lambda x: 1 if x == 'Stały związek' else 0)

df['Mieszkanie'] = df['Mieszkanie'].apply(lambda x: 1 if x == 'Własne' else 0)

df['Praca'] = df['Praca'].apply(lambda x: 1 if x == 'Umysłowy' else 0)

df['Ocena'] = df['Ocena'].apply(lambda x: 1 if x == 'Dobra' else 0)

In [111]:
df.isna().any()

Cel             False
Aktywa          False
Płeć            False
Stan cywilny    False
Mieszkanie      False
Praca           False
Okres           False
Wiek            False
Ocena           False
dtype: bool

In [112]:
df.head(10)

,Cel,Aktywa,Płeć,Stan cywilny,Mieszkanie,Praca,Okres,Wiek,Ocena
1,Drobne AGD,Bardzo niskie,1,0,1,0,13,23,1
2,Meble,Niskie,1,0,1,1,25,32,0
3,Nowy samochód,Bardzo niskie,1,0,1,1,19,38,0
4,Meble,Bardzo niskie,1,0,1,0,13,36,0
5,Edukacja,Niskie,1,0,0,1,40,31,1
6,Meble,Brak,1,1,1,1,11,25,1
7,Nowy samochód,Bardzo niskie,1,1,1,0,13,26,1
8,Business,Bardzo niskie,1,0,1,0,14,27,1
9,Drobne AGD,Bardzo niskie,1,0,1,1,37,25,0
10,Drobne AGD,Brak,0,0,1,1,25,43,0


In [113]:
df.iloc[:,2:-1].columns

Index(['Płeć', 'Stan cywilny', 'Mieszkanie', 'Praca', 'Okres', 'Wiek'], dtype='object')

In [114]:
y = df['Ocena']
y


1      1
2      0
3      0
4      0
5      1
      ..
421    1
422    0
423    0
424    0
425    1
Name: Ocena, Length: 425, dtype: int64

In [126]:
def hosmer_lemeshow(y, y_pred_prob, n_bins=10):
    y = np.array(y)
    y_pred_prob = np.array(y_pred_prob)

    # Tworzymy biny dla prognozowanych prawdopodobieństw
    percentiles = np.percentile(y_pred_prob, np.linspace(0, 100, n_bins + 1))
    bins = np.digitize(y_pred_prob, percentiles[1:-1])
    
    # Liczymy rzeczywiste i oczekiwane proporcje sukcesów
    actual = np.bincount(bins, weights=y) / np.bincount(bins)
    expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
    
    # Obliczamy statystykę testu Hosmera-Lemeshowa
    hl_stat = np.sum((actual - expected) ** 2 / (expected * (1 - expected) / np.bincount(bins)))
    
    # Obliczamy p-wartość
    p_value = stats.chi2.sf(hl_stat, n_bins - 2)
    
    return hl_stat, p_value

In [155]:
evaluation = pd.DataFrame({
    'Column' : [],
    'Std dev' : [],
    'AIC' : [],
    'BIC' : [],
    'R2_Nagelkera' : [],
    'Odds ratio' : [],
    'Hosmel test': [],
    'Significance': [],
})

y = df['Ocena']

for column in df.iloc[:,2:-1].columns:
    X = sm.add_constant(df[column])  # dodajemy stałą do modelu
    model = sm.Logit(y, X)
    results = model.fit()
    
    # Obliczanie p-wartości dla każdego parametru
    p_values = results.pvalues
    print(f"P-wartości dla {column}: ", p_values)
    
    # Odchylenie
    std_dev = results.bse[column]
    
    # Obliczanie AIC, BIC, R-kwadrat Nagelkera
    AIC = results.aic
    BIC = results.bic
    R2_Nagelkera = results.prsquared

    # Test Hosmera-Lemeshowa 
    hl_stat, p_value = hosmer_lemeshow(y, results.predict(X))
    print('Hosmer-Lemeshow test: H=%.3f, p=%.3f' % (hl_stat, p_value))
    
    # Iloraz szans
    odds_ratio = np.exp(results.params[column])
    
    print(AIC, BIC, R2_Nagelkera, odds_ratio)
    new_data = pd.DataFrame({
        'Column' : [column],
        'Std dev' : [std_dev],
        'AIC' : [AIC / 1000],
        'BIC' : [BIC / 1000],
        'R2_Nagelkera' : [R2_Nagelkera * 100],
        'Odds ratio' : [odds_ratio],
        'Hosmel test' : [hl_stat],
        'Significance': [p_values[1]],
    })    
    
    evaluation = pd.concat([evaluation, new_data], ignore_index=True)


Optimization terminated successfully.
         Current function value: 0.686949
         Iterations 4
P-wartości dla Płeć:  const    0.071860
Płeć     0.022628
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
587.9066530240073 596.010831361856 0.008906456482549618 1.615354174910962
Optimization terminated successfully.
         Current function value: 0.693013
         Iterations 3
P-wartości dla Stan cywilny:  const           0.959563
Stan cywilny    0.761094
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
593.0613644413224 601.1655427791712 0.00015711071308688318 1.1119155354449468
Optimization terminated successfully.
         Current function value: 0.683021
         Iterations 4
P-wartości dla Mieszkanie:  const         0.020086
Mieszkanie    0.003659
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
584.5679800436183 592.672158381467 0.01457335069867638 1.8551058620193004
Optimization terminated successfully.
         Current function value: 0.692955
         Iterations 3
P-wa

/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:11: RuntimeWarning: invalid value encountered in divide
  expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:11: RuntimeWarning: invalid value encountered in divide
  expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/f

In [156]:
evaluation

,Column,Std dev,AIC,BIC,R2_Nagelkera,Odds ratio,Hosmel test,Significance
0,Płeć,0.210363,0.587907,0.596011,0.890646,1.615354,NaN,0.022628
1,Stan cywilny,0.348910,0.593061,0.601166,0.015711,1.111916,NaN,0.761094
2,Mieszkanie,0.212633,0.584568,0.592672,1.457335,1.855106,NaN,0.003659
3,Praca,0.228849,0.593012,0.601116,0.024093,0.917414,NaN,0.706432
4,Okres,0.008601,0.571708,0.579813,3.640046,0.962479,NaN,0.000009
5,Wiek,0.008910,0.590049,0.598154,0.526942,1.015720,21.953839,0.079999
